In [1]:
import pandas as pd
import numpy as np
import re

DF_ideal_only = pd.read_csv("clean_data\idealwine_2020-11-21.csv")   # Données Idealwine sans jointure avec données RVF
DF_ideal_rvf0 = pd.read_csv("clean_data\idealwine_larvf_2020-11-26.csv")   # Données Idealwine jointes lorsque possible à celles de la RVF

# Suppression des "vieux" vins, trop rares pour faire l'objet d'une cote fiable :
DF_ideal_only = DF_ideal_only[DF_ideal_only.millesime>=1996]
DF_ideal_rvf0 = DF_ideal_rvf0[DF_ideal_rvf0.millesime>=1996]

DF_ideal_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38499 entries, 0 to 62040
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   nom_du_vin      38499 non-null  object 
 1   millesime       38499 non-null  int64  
 2   pays_region     38499 non-null  object 
 3   domaine         38499 non-null  object 
 4   appellation     38499 non-null  object 
 5   couleur         38499 non-null  object 
 6   cote_1992       0 non-null      float64
 7   cote_1993       0 non-null      float64
 8   cote_1994       0 non-null      float64
 9   cote_1995       0 non-null      float64
 10  cote_1996       0 non-null      float64
 11  cote_1997       1 non-null      float64
 12  cote_1998       40 non-null     float64
 13  cote_1999       611 non-null    float64
 14  cote_2000       1158 non-null   float64
 15  cote_2001       1300 non-null   float64
 16  cote_2002       1493 non-null   float64
 17  cote_2003       2079 non-null  

In [2]:
# Suppression des doublons dans la table jointe (lorsqu'un même vin a plusieurs notes RVF) :

L_col_ideal = ['nom_du_vin', 'millesime', 'pays_region', 'domaine', 'appellation', 'couleur', 'cote_1992', 'cote_1993', 'cote_1994', 'cote_1995', 'cote_1996', 'cote_1997', 'cote_1998', 'cote_1999', 'cote_2000', 'cote_2001', 'cote_2002', 'cote_2003', 'cote_2004', 'cote_2005', 'cote_2006', 'cote_2007', 'cote_2008', 'cote_2009', 'cote_2010', 'cote_2011', 'cote_2012', 'cote_2013', 'cote_2014', 'cote_2015', 'cote_2016', 'cote_2017', 'cote_2018', 'cote_2019', 'cote_2020', 'note_Burghound', 'note_JMQ', 'note_JR', 'note_VinousA', 'note_WA', 'note_WS']
L_col_rvf = ['rvf_domain_stars', 'rvf_review_score_float']
DF_ideal_rvf = DF_ideal_rvf0.groupby(L_col_ideal, dropna=False, as_index=False)[L_col_rvf].median()
DF_ideal_rvf.info()
# Même nombre de lignes que dans DF_ideal : 38499

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38499 entries, 0 to 38498
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nom_du_vin              38499 non-null  object 
 1   millesime               38499 non-null  int64  
 2   pays_region             38499 non-null  object 
 3   domaine                 38499 non-null  object 
 4   appellation             38499 non-null  object 
 5   couleur                 38499 non-null  object 
 6   cote_1992               0 non-null      float64
 7   cote_1993               0 non-null      float64
 8   cote_1994               0 non-null      float64
 9   cote_1995               0 non-null      float64
 10  cote_1996               0 non-null      float64
 11  cote_1997               1 non-null      float64
 12  cote_1998               40 non-null     float64
 13  cote_1999               611 non-null    float64
 14  cote_2000               1158 non-null 

In [4]:
# Préparation des données pour visualiser l'évolution de la cote d'un vin
L_col_id = ['pays_region', 'appellation', 'domaine', 'nom_du_vin', 'couleur', 'millesime']
L_col_cote = [col for col in DF_ideal_only.columns if ((re.search(r'^cote_', col)!=None) and (int(re.sub(r'^cote_', "", col))>=2000))]
DF_data = DF_ideal_only[DF_ideal_only.millesime>=1996][L_col_id + L_col_cote].reset_index(drop=True).reset_index()
DF_data = DF_data.melt(id_vars=['index']+L_col_id, value_vars=L_col_cote, var_name='annee_cote', value_name='cote')
DF_data['annee_cote'] = DF_data['annee_cote'].apply(lambda s: int(re.sub(r"^cote_", "", s)))
DF_data = DF_data.sort_values(by=['index', 'annee_cote']).dropna()
DF_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253505 entries, 692982 to 808478
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   index        253505 non-null  int64  
 1   pays_region  253505 non-null  object 
 2   appellation  253505 non-null  object 
 3   domaine      253505 non-null  object 
 4   nom_du_vin   253505 non-null  object 
 5   couleur      253505 non-null  object 
 6   millesime    253505 non-null  int64  
 7   annee_cote   253505 non-null  int64  
 8   cote         253505 non-null  float64
dtypes: float64(1), int64(3), object(5)
memory usage: 19.3+ MB


In [56]:
import plotly.express as px
import plotly.graph_objects as go

# Visualisation sur un vin :
name = "Beaujeu"
domain = "Cotat"
vintage = 2006
color = "Blanc"
DF_vin = DF_data[DF_data.nom_du_vin.apply(lambda s: (s.find(name)>-1)) & (DF_data.couleur==color) \
                 & DF_data.domaine.apply(lambda s: (s.find(domain)>-1))& (DF_data.millesime==vintage)]
print(len(DF_vin))
display(DF_vin.head(30))
title = DF_vin.nom_du_vin.tolist()[0] + " " + str(DF_vin.millesime.tolist()[0])

fig = px.scatter(DF_vin, x='annee_cote', y='cote', title=title)
fig.update_traces(mode="lines+markers", line=dict(color="#d41010"))
fig.update_traces(marker=go.scatter.Marker(size=10))
fig.update_layout(showlegend=False)
fig.update_xaxes(title_text="Année de cotation")
fig.update_yaxes(title_text="Valeur d'une bouteille de 75 cL en € (Idealwine)")
fig.show()


13


,index,pays_region,appellation,domaine,nom_du_vin,couleur,millesime,annee_cote,cote
320207,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2008,24.50
358706,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2009,24.50
397205,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2010,29.00
435704,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2011,32.00
474203,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2012,31.50
512702,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2013,33.50
551201,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2014,30.00
589700,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2015,34.00
628199,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2016,37.41
666698,12215,Vallée de la Loire,Sancerre,François Cotat,Sancerre Les Culs de Beaujeu François Cotat,Blanc,2006,2017,41.57
